In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

llm = ChatOpenAI(
    temperature=0.1,
)

def plus(a,b):
    return a+b

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=plus, 
            name="Sum calculator", 
            description = "Use this to perform sums of two numbers This tool take two arguments, both should be numbers",
        ),
    ]
)

prompt = "sum of, $355.39, $924.87, $721.2, $1940.29, $573.63, $65.72, $35.00, $552.00, $76.16, $29.12"
# $5,273.38
agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I need to calculate the sum of all the given numbers.
Action:
```
{
  "action": "Sum calculator",
  "action_input": {"a": 355.39, "b": 924.87}
}
```
Observation: 1280.26
Thought:```
{
  "action": "Sum calculator",
  "action_input": {"a": 1280.26, "b": 721.2}
}
```
Observation: 2001.46
Thought:```
{
  "action": "Sum calculator",
  "action_input": {"a": 2001.46, "b": 1940.29}
}
```
Observation: 3941.75
Thought:```
{
  "action": "Sum calculator",
  "action_input": {"a": 3941.75, "b": 573.63}
}
```
Observation: 4515.38
Thought:```
{
  "action": "Sum calculator",
  "action_input": {"a": 4515.38, "b": 65.72}
}
```
Observation: 4581.1
Thought:```
{
  "action": "Sum calculator",
  "action_input": {"a": 4581.1, "b": 35.00}
}
```
Observation: 4616.1
Thought:```
{
  "action": "Sum calculator",
  "action_input": {"a": 4616.1, "b": 552.00}
}
```
Observation: 5168.1
Thought:```
{
  "action": "Sum calculator",
  "action_input": {"a": 5168.1, "b": 76.16

{'input': 'sum of, $355.39, $924.87, $721.2, $1940.29, $573.63, $65.72, $35.00, $552.00, $76.16, $29.12',
 'output': 'The sum of $355.39, $924.87, $721.2, $1940.29, $573.63, $65.72, $35.00, $552.00, $76.16, $29.12 is $5273.38'}